In [124]:
from coincidence_matching import match_parts, get_export_id_types, match_parts_dict
from zipfile import ZipFile
import json
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from automate import Part
import numpy as np
import meshplot as mp
from collections import Counter
import torch

from brepmatching.data import BRepMatchingDataset

In [2]:
import os
from pathlib import Path
str(Path.home().joinpath('.config','onshapecreds.json'))

'/homes/grail/benjones/.config/onshapecreds.json'

In [3]:
gdrive_f1 = "https://drive.google.com/file/d/1O7FajIOF5YT_1zW337pm6XE_B2gEhUUc"
gdrive_f2 = "https://drive.google.com/file/d/1OK4qyuUfSXqA-wd64p_f2pG0rxeeqXGM"
gdrive_f3 = "https://drive.google.com/file/d/1NnUBCkIUrYBTXvEC8IXGE4V2UNDTG6T6"

In [6]:
def gdrive_wget(FILEID, FILENAME):
    return f"wget --load-cookies /tmp/cookies.txt \"https://docs.google.com/uc?export=download&confirm=" + \
        "$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate" + \
        " 'https://docs.google.com/uc?export=download&id={FILEID}' -O- | sed -rn 's/.*confirm=([0-9A-" + \
        "Za-z_]+).*/\1\n/p')&id={FILEID}\" -O {FILENAME} && rm -rf /tmp/cookies.txt"
def url_to_wget(url, dest):
    url = url.split('/d/')[-1]
    return gdrive_wget(url, dest)

In [ ]:
from zipfile import ZipFile
import pandas as pd
from tqdm import tqdm
from apikey.onshape import Onshape
from pathlib import Path

api = Onshape(stack='https://cad.onshape.com', creds=str(Path.home().joinpath('.config','onshapecreds.json')), logging=False)

with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    variations[:100]
    variations = variations[variations.fail == 0]

    failures = []
    written = set()
    with ZipFile('../data/redownload.zip','w') as wzf:
        for i in tqdm(range(len(variations))):
            d = variations.iloc[i]

            response_orig = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_orig}/e/{d.eid}/parasolid', query={'includeExportIds':True})
            response_var = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})

            if response_orig.status_code != 200 or response_var.status_code != 200:
                failures.append(d)
                continue
            
            orig_path = 'data/BrepsWithReference/' + d.ps_orig
            var_path = 'data/BrepsWithReference/' + d.ps_var

            if orig_path not in written:
                with wzf.open(orig_path, 'w') as f:
                    f.write(response_orig.text.encode('utf-8'))
                written.add(orig_path)
            if var_path not in written:
                with wzf.open(var_path, 'w') as f:
                    f.write(response_var.text.encode('utf-8'))
                written.add(var_path)

In [13]:
os.listdir('../../../brepmatching/Geo.zip')

['Topo.zip',
 'TopoAndGeoRedownload.zip',
 'Geo.zip',
 'TopoRedownload.zip',
 'TopoAndGeo.zip',
 'GeoRedownload.zip',
 'GeoRedownload.zip-failures.pickle']

In [14]:
import pickle
with open('../../../brepmatching/GeoRedownload.zip-failures.pickle','rb') as f:
    failures = pickle.load(f)
    

In [18]:
failures[0].link_var

'https://cad.onshape.com/documents/13d0516ffa3e9971c6cf76ab/w/f4cf14813a56fd6239cf9094/m/4ca0d1a308e655e35faed811/e/d4d5e78afdef61847ae053bf'

In [23]:
zf =  ZipFile('../../../brepmatching/Geo.zip','r') 

In [34]:
with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
    variations = pd.read_csv(f)

ValueError: Attempt to use ZIP archive that was already closed

In [25]:
allnames = set(zf.namelist())

In [26]:
zf.close()

In [77]:
var_exists = []
missing = []
for p in 'data/baseline/' + variations[variations.fail == 0].baselineNew:
    if not p in allnames:
        missing.append(p)
len(missing)

9

In [78]:
missing

['data/baseline/ ',
 'data/baseline/baselineNew_a11220c88990f369f613901b_4302ae6f3b025356bf93a4ea_ed40c2263fb2591d5c32014f_default_jjieiV3.x_t',
 'data/baseline/baselineNew_5820f8088ae1e8103b2bf77b_a30fa1a47133f01cd90768f9_cfbc6fa11e088963e6f40230_default_jjdeqV3.x_t',
 'data/baseline/baselineNew_a2eb8f3d2f3907ce4262b2a9_4ff8e3e6143517bbb153119a_f5de5ac33eb7c5717fa5ef6f_default_jjeeiV1.x_t',
 'data/baseline/baselineNew_5885d6d43a9ead102a8c5ca9_fd4b966909e746c8e86bbf1d_5d2846a7d52c0da912358721_default_jjeeiV1.x_t',
 'data/baseline/baselineNew_28f4025615c0c9f3cd89f3f4_4fae74bb3d3c325aa7ac2925_1ff3a232438852bc665ceb65_default_jjeeiV2.x_t',
 'data/baseline/baselineNew_ff226acc53b2673250e3ba83_26b3f6ccc587cdf48c287835_f45e414a58f2a21fec19b385_default_jjdeiV2.x_t',
 'data/baseline/baselineNew_504843aac699bc2fccd7d661_8c6f274019fbaccf30a6f171_78b2cdc2a36cbb4eaafb374d_default_jjeeiV1.x_t',
 'data/baseline/baselineNew_cc8d096b3a7badb0eab93cd5_4f795c6f7e39bd0dda0bfacf_8aa8948806f4b554a33dcfba_de

In [71]:
len(variations[variations.fail == 0])

6798

In [72]:
variations.baselineOrig.iloc[1]

'baselineOrig_d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf82d0_5dcc8a4afa039812c247cda8_default_jjeeiV2.x_t'

In [74]:
'data/baseline/baselineOrig_d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf82d0_5dcc8a4afa039812c247cda8_default_jjeeiV2.x_t' in allnames

True

In [160]:
not_failed = variations[variations.fail == 0]

In [161]:
not_failed[not_failed.baselineNew == ' ']

,name,did,wid,eid,link_orig,mv_orig,ps_orig,mv_var,ps_var,matchFile,codeVersion,fail,seed,link_var,baselineOrig,baselineNew,baselineMatch,translationFail
0,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,8f868ab8ff16bbca8df8f914,1be1aa77cc97fec37f31922c,b459723517ecd2dbff382c8e,https://cad.onshape.com/documents/8f868ab8ff16...,2e43732de79d2a8bf3a1bac6,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,ce6be646700055037d4f196d,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,MCV2,0,10472,https://cad.onshape.com/documents/8f868ab8ff16...,,,,0


In [147]:
missing_data = []
has_missing_export_id = []
missing_match_count = []
has_missing_matches = []
with ZipFile('../../../brepmatching/Geo.zip','r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    
    data = variations
    data = data[(data.fail == 0) & (data.baselineNew != ' ') ]
    
    all_names = set(zf.namelist())
    
    for i in tqdm(range(100)):#tqdm(range(len(data))):
        d = data.iloc[i]
        
        original_path = 'data/BrepsWithReference/' + d.ps_orig
        variation_path = 'data/BrepsWithReference/' + d.ps_var
        
        baseline_original_path = 'data/baseline/' + d.baselineOrig
        baseline_var_path = 'data/baseline/' + d.baselineNew
        
        path_gt_match = 'data/Matches/' + d.matchFile
        path_bl_match = 'data/baseline/' + d.baselineMatch
        
        data_exists = (original_path in all_names) and \
            (variation_path in all_names) and \
            (baseline_original_path in all_names) and \
            (baseline_var_path in all_names) and \
            (path_gt_match in all_names) and \
            (path_bl_match in all_names)
        
        # Verify that all data files are in the archive
        if not data_exists:
            missing_data.append(i)
            continue
        
        # Load parts and match files
        with zf.open(original_path,'r') as f:
            original_part_data = f.read().decode('utf-8')
        with zf.open(variation_path,'r') as f:
            variation_part_data = f.read().decode('utf-8')
        with zf.open(baseline_original_path, 'r') as f:
            baseline_original_data = f.read().decode('utf-8')
        with zf.open(baseline_var_path, 'r') as f:
            baseline_var_data = f.read().decode('utf-8')
        with zf.open(path_gt_match, 'r') as f:
            gt_match = json.load(f)
        with zf.open(path_bl_match, 'r') as f:
            baseline_match = json.load(f)
            
        
        original_ids = get_export_id_types(original_part_data)
        variation_ids = get_export_id_types(variation_part_data)
        
        bl_original_ids = get_export_id_types(baseline_original_data)
        bl_variation_ids = get_export_id_types(baseline_var_data)
        
        # Verify that all referenced export_ids exist in the corresponding files
        export_id_missing = False
        for v in gt_match.values():
            orig_export_id = v['val1']
            var_export_id = v['val2']
            
            if orig_export_id not in original_ids or var_export_id not in variation_ids:
                export_id_missing = True
        
        for v in baseline_match.values():
            orig_export_id = v['val1']
            var_export_id = v['val2']
            
            if orig_export_id not in bl_original_ids or var_export_id not in bl_variation_ids:
                export_id_missing = True
        
        if export_id_missing:
            has_missing_export_id.append(i)
            continue
        
        # Check that exact matching the baseline variation to the primary variation always works
        
        var_matching = match_parts(baseline_var_data, variation_part_data, True)
        var_matching_dict = match_parts_dict(baseline_var_data, variation_part_data, True)
        
        n_faces_variation = len([k for k,v in variation_ids.items() if v == 'PK_CLASS_face'])
        n_faces_baseline_variation = len([k for k,v in bl_variation_ids.items() if v == 'PK_CLASS_face'])
        
        n_edges_variation = len([k for k,v in variation_ids.items() if v == 'PK_CLASS_edge'])
        n_edges_baseline_variation = len([k for k,v in bl_variation_ids.items() if v == 'PK_CLASS_edge'])
        
        n_vertices_variation = len([k for k,v in variation_ids.items() if v == 'PK_CLASS_vertex'])
        n_vertices_baseline_variation = len([k for k,v in bl_variation_ids.items() if v == 'PK_CLASS_vertex'])
        
        #assert(len(var_matching.face_matches) == n_faces_variation)
        #assert(len(var_matching.face_matches) == n_faces_baseline_variation)
        #assert(len(var_matching.edge_matches) == n_edges_variation)
        #assert(len(var_matching.edge_matches) == n_edges_baseline_variation)
        #assert(len(var_matching.vertex_matches) == n_vertices_variation)
        #assert(len(var_matching.vertex_matches) == n_vertices_baseline_variation)
        
        good_topos = ['PK_CLASS_face', 'PK_CLASS_edge', 'PK_CLASS_vertex']
        
        n_matches_missing = 0
        for v in baseline_match.values():
            var_export_id = v['val2']
            if bl_variation_ids[var_export_id] in good_topos:
                if var_export_id not in var_matching_dict:
                    n_matches_missing += 1
        
        missing_match_count.append(n_matches_missing)
        
        if n_matches_missing > 0:
            has_missing_matches.append(i)
            
        if n_matches_missing > 2:
            break
    

 67%|███████████████████████████████████████████████████████████████▋                               | 67/100 [01:03<00:31,  1.05it/s]


In [156]:
len(missing_data)

1

In [150]:
variation_part = Part(variation_part_data)
baseline_variation_part = Part(baseline_var_data)

In [152]:
mp.plot(variation_part.mesh.V, variation_part.mesh.F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [131]:
n_edges_baseline_variation

24

In [133]:
len(var_matching.edge_matches)

23

In [102]:
original_export_types = get_export_id_types(original_part_data)

In [103]:
match_ids_original = [v['val1'] for v in gt_match.values()]
match_ids_variation = [v['val2'] for v in gt_match.values()]
bl_match_ids_original = [v['val1'] for v in baseline_match.values()]
bl_match_ids_variation = [v['val2'] for v in baseline_match.values()]



In [107]:
all([x in original_export_types for x in match_ids_original])

True

In [98]:
for v in gt_match.values():
    orig_export_id = v['val1']
    var_export_id = v['val2']

    if orig_export_id not in original_part_data:
        print(f'missing orig: {orig_export_id}')
    
    if var_export_id not in variation_part_data:
        print(f'missing var: {var_export_id}')

for v in baseline_match.values():
    bl_orig_export_id = v['val1']
    bl_var_export_id = v['val2']

    if bl_orig_export_id not in baseline_original_data:
        print(f'missing baseline orig: {bl_orig_export_id}')
    
    if bl_var_export_id not in baseline_var_data:
        print(f'missing baseline var {bl_var_export_id}')

missing var: KF4C
missing orig: KFcD
missing var: KFsE
missing orig: KF8E
missing var: KJBD
missing orig: JF9
missing orig: KFdD
missing var: KFJG
missing orig: KFhG
missing var: KF9G
missing var: KFZH
missing var: KFJK
missing orig: KFlL
missing var: KFlL
missing var: KJBB
missing var: KI2C
missing orig: JF2
missing var: KIaD
missing var: KIKB
missing var: KIaB
missing var: KIqB
missing var: KI6B
missing var: KIKC
missing var: KFGE
missing var: KFaE
missing var: KI+D
missing baseline var KF8B
missing baseline orig: KFkD
missing baseline orig: KFME
missing baseline var KFRD
missing baseline orig: KFVD
missing baseline var KFRE
missing baseline var KFFF
missing baseline var KFtF
missing baseline orig: KF9F
missing baseline var KFJG
missing baseline var KFdG
missing baseline orig: KF9G
missing baseline orig: KFNH
missing baseline orig: KFBI
missing baseline orig: KFVI
missing baseline orig: KFJJ
missing baseline var KFRJ
missing baseline var KFxK
missing baseline orig: KFNM
missing basel

In [22]:
#with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:

#with ZipFile('../data/GeoV2FullRunBaseline.zip','r') as zf:

with ZipFile('../../../brepmatching/Geo.zip','r') as zf:


    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)

    data = variations[:100]
    data = data[(data.fail == 0) & (data.baselineNew != ' ') ]


#with ZipFile('../../../brepmatching/GeoRedownload.zip','r') as zf:
    
    incomplete = []
    incompatible = []
    diff_geo = []

    m_f = []
    m_e = []
    m_v = []

    A = []
    B = []
    M = []

    for i in tqdm(range(len(data))):
        d = data.iloc[i]
        path_a = 'data/BrepsWithReference/' + d.ps_orig
        path_b = 'data/baseline/' + d.baselineOrig

        path_c = 'data/BrepsWithReference/' + d.ps_var
        path_d = 'data/baseline/' + d.baselineNew
        
        path_gt_match = 'data/Matches/' + d.matchFile
        path_bl_match = 'data/baseline/' + d.baselineMatch

        with zf.open(path_a,'r') as f:
            part_data_a = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_b = f.read().decode('utf-8')
        with zf.open(path_c,'r') as f:
            part_data_c = f.read().decode('utf-8')
        with zf.open(path_d,'r') as f:
            part_data_d = f.read().decode('utf-8')
        with zf.open(path_gt_match,'r') as f:
            gt_match = json.load(f)
        with zf.open(path_bl_match, 'r') as f:
            bl_match = json.load(f)

        p_a = Part(part_data_a)
        p_b = Part(part_data_b)
        p_c = Part(part_data_c)
        p_d = Part(part_data_d)

        parts_have_same_num = len(p_a.brep.nodes.faces) == len(p_b.brep.nodes.faces) and len(p_a.brep.nodes.edges) == len(p_b.brep.nodes.edges) and len(p_a.brep.nodes.vertices) == len(p_b.brep.nodes.vertices)

        matching = match_parts(part_data_a, part_data_b, True)
        complete_matching = len(matching.face_matches) == len(p_a.brep.nodes.faces) and len(matching.edge_matches) == len(p_a.brep.nodes.edges) and len(matching.vertex_matches) == len(p_a.brep.nodes.vertices)

        n_face_matches_missing = -len(matching.face_matches) + len(p_a.brep.nodes.faces)
        n_edge_matches_missing = -len(matching.edge_matches) + len(p_a.brep.nodes.edges)
        n_vertex_matches_missing =-len(matching.vertex_matches) + len(p_a.brep.nodes.vertices)

        m_f.append(n_face_matches_missing)
        m_e.append(n_edge_matches_missing)
        m_v.append(n_vertex_matches_missing)

        exact_geo = (p_a.mesh.V == p_b.mesh.V).all()

        if not parts_have_same_num:
            incompatible.append(i)
        if not complete_matching:
            incomplete.append(i)

        if not exact_geo:
            diff_geo.append(i)

        A.append(p_a)
        B.append(p_b)
        M.append(matching)


        # Exact matching of originals seems to work (and the tesselations are mostly identical!)
        # Let's see how well we match the versions with variations:

        var_matching = match_parts(part_data_c, part_data_d, True)

        id_classes_a = get_export_id_types(part_data_a)
        id_classes_b = get_export_id_types(part_data_b)
        id_classes_c = get_export_id_types(part_data_c)
        id_classes_d = get_export_id_types(part_data_d)

        good_classes = ['PK_CLASS_face', 'PK_CLASS_edge', 'PK_CLASS_vertex']

        #gt_topo_matches = [(v['val1'],v['val1']) for k,v in gt_match.items() if id_classes_a[v['val1']] in good_classes and id_classes_c[v['val2']] in good_classes]
        bl_topo_matches = [(v['val1'],v['val1']) for k,v in bl_match.items() if id_classes_b[v['val1']] in good_classes and id_classes_d[v['val2']] in good_classes]

        


 62%|███████████████████████████████████████████████████████████▏                                    | 61/99 [00:21<00:13,  2.78it/s]


KeyError: "There is no item named 'data/baseline/baselineOrig_a11220c88990f369f613901b_4302ae6f3b025356bf93a4ea_ed40c2263fb2591d5c32014f_default_jjieiV3.x_t' in the archive"

In [ ]:
'JF4' in part_data_d

In [ ]:
data

In [ ]:
d.link_var

In [ ]:
part_b = Part(part_data_b)

In [ ]:
len([e.export_id for e in part_b.brep.nodes.edges])

In [ ]:
len([k for k,v in id_classes_b.items() if v == 'PK_CLASS_edge'])

In [ ]:
id_classes_b

In [ ]:
d.link_var

In [ ]:
data.baselineMatch

In [ ]:
from apikey.onshape import Onshape

In [ ]:
response = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})#, query={'partIds':['JFD'],'includeExportIds':True})

In [ ]:
response.text == part_data_b

In [ ]:
d.mv_var

In [ ]:
d.did

In [ ]:
d.eid

In [ ]:
id_classes_b